<a href="https://colab.research.google.com/github/emanuelebrizzi/bootcamp/blob/main/code_vulnerability_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 21.6 MB/s eta 0:00:00


In [ ]:
#from google.colab import files
#import zipfile
#import os

from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


# **Dataset upload**


In [ ]:
import os
import glob

# Directory path
drive_path = "/content/drive/MyDrive/bootcamp_file"

libpng_path = os.path.join(drive_path, "LibPNG")

vuln_path = os.path.join(libpng_path, "Vulnerable_functions")
non_vuln_path = os.path.join(libpng_path, "Non_vulnerable_functions")

In [ ]:

def load_files_from_folder(folder, label):
    files = glob.glob(os.path.join(folder, "*.c"))
    data = []
    for file in files:
        with open(file, "r", encoding="utf-8", errors="ignore") as f:
            data.append((f.read(), label, os.path.basename(file)))
    return data


# **Replace Function names and Variable names**

In [ ]:
import re
import random
import string
from pathlib import Path

In [ ]:
def remove_comments(code):
    """Rimuove i commenti dal codice C."""
    code = re.sub(r'//.*', '', code)  # Rimuove commenti su singola riga
    code = re.sub(r'/\*.*?\*/', '', code, flags=re.DOTALL)  # Rimuove commenti multi-linea
    return code

def generate_random_name(length=8):
    """Genera un nome casuale per variabili e funzioni."""
    return ''.join(random.choices(string.ascii_letters, k=length))

def extract_identifiers(code):
    """Estrae nomi di variabili e funzioni dal codice C."""
    pattern = re.compile(r'\b([a-zA-Z_][a-zA-Z0-9_]*)\b')
    keywords = set(['int', 'char', 'float', 'double', 'return', 'if', 'else', 'while', 'for', 'do', 'switch', 'case', 'void'])
    identifiers = set(pattern.findall(code)) - keywords
    return identifiers

def categorize_identifiers(identifiers):
    """Classifica gli identificatori come variabili o funzioni."""
    var_counter, fun_counter = 1, 1
    replacements = {}
    for identifier in identifiers:
        if re.search(r'\b[A-Za-z_][A-Za-z0-9_]*\s*\(', identifier):
            replacements[identifier] = f'fun_{fun_counter}'
            fun_counter += 1
        else:
            replacements[identifier] = f'var_{var_counter}'
            var_counter += 1
    return replacements

def replace_identifiers(code, replacements):
    """Sostituisce i nomi di variabili e funzioni con nuovi nomi."""
    for old, new in replacements.items():
        code = re.sub(r'\b' + re.escape(old) + r'\b', new, code)
    return code

def process_c_file(filepath, new_filepath):
    """Legge un file C, sostituisce i nomi e salva il nuovo file."""
    with open(filepath, 'r', encoding='utf-8') as file:
        code = file.read()

    code_no_comments = remove_comments(code)
    identifiers = extract_identifiers(code_no_comments)
    replacements = categorize_identifiers(identifiers)
    new_code = replace_identifiers(code_no_comments, replacements)
    with open(new_filepath, 'w', encoding='utf-8') as file:
        file.write(new_code)

def process_directory(directory, new_directory_path):
    """Processa tutti i file C in una directory."""
    for filepath in Path(directory).glob("*.c"):
        new_filepath = os.path.join(new_directory_path, os.path.basename(filepath))
        process_c_file(filepath, new_filepath)

Execution:

In [ ]:
import shutil

modified_vuln_path = os.path.join(vuln_path, "modified_files")

if os.path.exists(modified_vuln_path):
  shutil.rmtree(modified_vuln_path)
os.makedirs(modified_vuln_path, exist_ok=True)
process_directory(vuln_path, modified_vuln_path)


modified_non_vuln_path = os.path.join(non_vuln_path, "modified_files")

if os.path.exists(modified_non_vuln_path):
  shutil.rmtree(modified_non_vuln_path)
os.makedirs(modified_non_vuln_path, exist_ok=True)
process_directory(non_vuln_path, modified_non_vuln_path)

# **Tokenization**

Using CodeBERT tokenizer

In [ ]:
!pip install transformers datasets torch scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 75.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 59.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 40.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
from transformers import RobertaTokenizer, RobertaForSequenceClassification, RobertaConfig
from datasets import Dataset, load_dataset, DatasetDict
import torch


# Load CodeBERT tokenizer
tokenizer = RobertaTokenizer.from_pretrained("microsoft/codebert-base")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/498 [00:00<?, ?B/s]

Preparing the Dataset

In [ ]:
def load_data_from_directory(directory, label):
    """Load C files from a directory and assign labels."""
    data = []
    for filename in os.listdir(directory):
        if filename.endswith(".c"):  # Ensure only C files are processed
            with open(os.path.join(directory, filename), "r", encoding="utf-8") as f:
                data.append({"text": f.read(), "label": label})
    return data

# Load data
vuln_data = load_data_from_directory(modified_vuln_path, 1)
non_vuln_data = load_data_from_directory(modified_non_vuln_path, 0)

# Combine datasets
dataset = vuln_data + non_vuln_data

# Convert to Hugging Face Dataset
dataset = Dataset.from_list(dataset)

# Tokenization function
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)

# Tokenize dataset
tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Split dataset into train & test
split_dataset = tokenized_datasets.train_test_split(test_size=0.2)


Map:   0%|          | 0/622 [00:00<?, ? examples/s]

In [13]:
from transformers import TrainingArguments, Trainer

# Load CodeBERT model for classification
model = RobertaForSequenceClassification.from_pretrained("microsoft/codebert-base", num_labels=2)

# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=split_dataset["train"],
    eval_dataset=split_dataset["test"],
    tokenizer=tokenizer,
)

# Train the model
trainer.train()

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at microsoft/codebert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-13-21aaffcd2c19>:20: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:


Abort: 

In [ ]:
# Evaluate model
metrics = trainer.evaluate()
print(metrics)

# Save model & tokenizer
model.save_pretrained("fine_tuned_codebert")
tokenizer.save_pretrained("fine_tuned_codebert")
